Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

# Data processing through MapReduce

![MapReduce](http://mm-tom.s3.amazonaws.com/blog/MapReduce.png)

## Vector norm
We want toompute the vector norm $|v| = \sqrt(\sum_i v_i^2)$ with a Map-Reduce process:
- use `map` function
- use `reduce` funtion from `functools`

- The `map(func, seq)` Python function applies the function func to all the elements of the sequence seq. It returns a new list with the elements changed by func

- The function `reduce(func, seq)` continually applies the function func() to the sequence seq and return a single value. For example, reduce(f, [1, 2, 3, 4, 5]) calculates f(f(f(f(1,2),3),4),5).

### Exercise 1.1
Write these two functions and compute the norm of V representing by the Python list above.

In [8]:
V = [4,1,2,3]

## Wordcount Example

[WordCount](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html#Example:_WordCount_v1.0) is a simple application that counts the number of occurrences of each word in a given input set.

Each mapper takes a line of text files as input and breaks it into words. It then emits a key/value pair of the word and 1 (separated by a tab). Each reducer sums the counts for each word and emits a single key/value with the word and sum.

In [34]:
from lorem import text
t = text()

with open("sample.txt", "w") as sample:
    sample.write(t)

print(t[:500]) # print only 500 first characters

Aliquam etincidunt aliquam amet eius sed modi. Consectetur quiquia eius dolore quiquia velit. Amet ut aliquam etincidunt tempora velit. Quiquia consectetur magnam eius. Labore velit non est.

Numquam quisquam eius sit neque tempora neque. Amet quisquam amet etincidunt. Consectetur porro eius eius labore. Tempora ipsum ut neque dolore quiquia aliquam amet. Sit magnam numquam adipisci quaerat labore eius dolorem. Labore aliquam neque dolor etincidunt velit aliquam. Aliquam labore est sit neque. Do


### Exercise 1.2
Write a python program that counts the number of words in that file.


## Read file and return a key/value pairs

### Exercise 1.3

Write a function `words` with file name as input that returns a sorted sequence of (word, 1) values.

Hints: `str.lower` , `str.maketrans` and `str.translate` methods can help to remove punctuation (`string.punctuation`).

# Reduce 

### Exercice 1.4

Write the funtion `reduce` to read the results of words and sum the occurrences of each word to a final count, and then output the results
as a list of (word, occurences). Two steps:
- Group (word, 1) pairs into a dictionary as
```python
{word1 : [1, 1], word2 : [1, 1, 1], word3 : [1] }
```
- Reduce operation prints out the word and its number of occurences.

### Exercise 1.5

Use `words` and `reduce` functions to return the words list of sample.txt with number of occurrences.

Each item of this list is a tuple. Sort this list with `operator.itemgetter(1)` to use the second element of this tuple as key for the `sorted` function.

### Exercise 1.6

Create 8 files like `sample0-7.txt` and use functions implemented above to
count word occurences. Most common word at the top of the output list.